In [1]:
import os 
import json
import logging
import pandas as pd 
from tqdm import tqdm

import requests
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
def get_FUND(url):
    FUND = {}
    
#     url = 'http://fund.hncb.com.tw/w/wb/wb02.djhtm?a=JAZ02-2001&B=0&C=2021-8-1&D=2022-2-28'
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get(url)
    sleep(5)
    
    select = driver.find_element(by=By.NAME, value='selFund3')
    FUND['fund_code'] = [x.get_attribute('value').split('-')[0] for x in select.find_elements(by=By.TAG_NAME, value='option')]
    FUND['fund_name'] = [x.text.split(' ')[1] for x in select.find_elements(by=By.TAG_NAME, value='option')]
    
    FUND = pd.DataFrame(FUND)
    
    driver.close()
    return FUND

In [3]:
def get_NETVALUE(FUND):
    NETVALUE = {}
    count = 0
    
    for FUND_idx in tqdm(range(len(FUND))):
        url = 'http://fund.hncb.com.tw/w/bcd/BCDNavList.djbcd?a={}&b=0&c=2021-8-1&d=2022-2-28'.format(FUND['fund_code'][FUND_idx])
        response = requests.get(url)
        data = response.text
        
        try:
        
            date = data.split(' ')[0].split(',')
            netvalue = data.split(' ')[1].split(',')
            assert len(date) == len(netvalue)

            for data_idx in range(len(date)):
                NETVALUE[count] = {'基金號碼': FUND['fund_code'][FUND_idx], 
                                   '基金名稱': FUND['fund_name'][FUND_idx], 
                                   '年月日': date[data_idx], 
                                   '淨值（美元）': netvalue[data_idx]}
                count+=1
            sleep(3)

        except:
            print(FUND_idx)
    return NETVALUE

In [4]:
url = 'http://fund.hncb.com.tw/w/wb/wb02.djhtm?a=SBZD7-1207&B=0&C=2021-8-1&D=2022-2-28'

FUND = get_FUND(url)
NETVALUE = get_NETVALUE(FUND)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/kasane/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/Users/kasane/Documents/Master/Class/1-2/Big data application project/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """
100%|███████████████████████████████████████████| 41/41 [02:10<00:00,  3.18s/it]


In [5]:
data = pd.DataFrame(NETVALUE).T

In [6]:
data.to_excel('../data/瑞銀淨值.xlsx', index=None)

In [7]:
df = pd.read_excel('../data/瑞銀淨值.xlsx')

In [8]:
df.head()

,基金號碼,基金名稱,年月日,淨值（美元）
0,SBZD7,瑞士銀行(瑞士)歐洲機會基金(未申報生效),20210802,928.01
1,SBZD7,瑞士銀行(瑞士)歐洲機會基金(未申報生效),20210803,932.13
2,SBZD7,瑞士銀行(瑞士)歐洲機會基金(未申報生效),20210804,937.61
3,SBZD7,瑞士銀行(瑞士)歐洲機會基金(未申報生效),20210805,941.86
4,SBZD7,瑞士銀行(瑞士)歐洲機會基金(未申報生效),20210806,942.91
